In [1]:
import torch
import sys
sys.path.append('../')

In [2]:
import os
import torch
import torch.distributed as dist
from torch import nn
from torch.nn.parallel import DistributedDataParallel as DDP

import numpy as np
import wandb

from tqdm import tqdm
from utils import accuracy
from datasets import set_datasets
from ddp_functions import setup, cleanup, set_dataloader
from train_functions import train, inference, set_optimizer, set_lr_scheduler
from utils import get_lr, color_print
from CAWR import CosineAnnealingWarmUpRestarts
from models import set_model
from args import set_parser

import torch.backends.cudnn as cudnn
cudnn.benchmark = True

from datetime import datetime
from pytz import timezone


In [3]:
train_dataset, _, _ = set_datasets(224)

In [4]:
train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,
        batch_size=11,
        num_workers=4,
        pin_memory=True,
    )

In [5]:
model = set_model('dinov2', rank = 'cuda', learning_method = 'classifier').cuda()

Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


In [6]:
from tqdm import tqdm

In [7]:
model.train()
loss_function = torch.nn.CrossEntropyLoss().cuda()

output_list = list()
label_list = list()
acc_list = list()
loss_list = list()


for data in tqdm(train_loader):
    image, label = data
    image = image.cuda()
    label = label.cuda()
    batch_size = label.shape[0]

    output = model(image)

    loss = loss_function(output, label)
    acc = accuracy(output, label)

    output_list.append(output.detach())
    label_list.append(label)
    acc_list.append(batch_size * acc)
    loss_list.append(batch_size * loss)

100%|██████████| 364/364 [00:05<00:00, 72.32it/s]


In [17]:
torch.stack(loss_list).shape

torch.Size([364])